In [20]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop columns that don't belong in the model
application_df = application_df.drop(["EIN","NAME"],axis=1)

In [21]:
# Determine unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [22]:
# Bin APPLICATION_TYPE value counts 
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [23]:
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [24]:
# Bin CLASSIFICATION value counts 
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [25]:
class_counts[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [26]:
class_counts[class_counts > 10]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: count, dtype: int64

In [27]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [28]:
application_with_dummies_df = pd.get_dummies(application_df)

In [29]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                1569040   
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1571501 (5.99 MB)
Trainable params: 1571501 (5.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 17s 19ms/step - loss: 0.4957 - accuracy: 0.7553
Epoch 2/100
804/804 [==============================] - 16s 19ms/step - loss: 0.1330 - accuracy: 0.9524
Epoch 3/100
804/804 [==============================] - 17s 21ms/step - loss: 0.0978 - accuracy: 0.9606
Epoch 4/100
804/804 [==============================] - 22s 28ms/step - loss: 0.0945 - accuracy: 0.9617
Epoch 5/100
804/804 [==============================] - 21s 26ms/step - loss: 0.0925 - accuracy: 0.9619
Epoch 6/100
804/804 [==============================] - 22s 27ms/step - loss: 0.0921 - accuracy: 0.9612
Epoch 7/100
804/804 [==============================] - 19s 24ms/step - loss: 0.0912 - accuracy: 0.9625
Epoch 8/100
804/804 [==============================] - 18s 22ms/step - loss: 0.0900 - accuracy: 0.9624
Epoch 9/100
804/804 [==============================] - 19s 24ms/step - loss: 0.0894 - accuracy: 0.9634
Epoch 10/100
804/804 [==============================] - 23s 29ms/step - l

804/804 [==============================] - 20s 25ms/step - loss: 0.0699 - accuracy: 0.9715
Epoch 80/100
804/804 [==============================] - 19s 24ms/step - loss: 0.0696 - accuracy: 0.9720
Epoch 81/100
804/804 [==============================] - 19s 23ms/step - loss: 0.0696 - accuracy: 0.9718
Epoch 82/100
804/804 [==============================] - 21s 26ms/step - loss: 0.0695 - accuracy: 0.9716
Epoch 83/100
804/804 [==============================] - 22s 28ms/step - loss: 0.0694 - accuracy: 0.9720
Epoch 84/100
804/804 [==============================] - 18s 22ms/step - loss: 0.0691 - accuracy: 0.9719
Epoch 85/100
804/804 [==============================] - 18s 22ms/step - loss: 0.0689 - accuracy: 0.9721
Epoch 86/100
804/804 [==============================] - 21s 26ms/step - loss: 0.0701 - accuracy: 0.9715
Epoch 87/100
804/804 [==============================] - 19s 23ms/step - loss: 0.0686 - accuracy: 0.9724
Epoch 88/100
804/804 [==============================] - 21s 26ms/step - loss:

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

# Extra - for keeps

In [ ]:
from tensorflow.keras.models import load_model

nn_reloaded = load_model('AlphabetSoupCharity.h5')

In [ ]:
model_loss, model_accuracy = nn_reloaded.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")